### CIFAR-10 using Convoluted Neural Network (CNN)

In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')

# Load MNIST dataset using TensorFlow
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using device: mps


If "MPS" appears in the output, then it utilizes Apple Silicon GPU.

In [2]:
# Normalize the data
x_train, x_test = x_train.astype(np.float32) / 255.0, x_test.astype(np.float32) / 255.0

# Create a custom dataset class
class CIFAR10Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10Dataset(x_train, y_train, transform=transform)
test_dataset = CIFAR10Dataset(x_test, y_test, transform=transform)

# Define data laoders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# Define the CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)

    def forward(self, x):
        x = self.pool(self.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(self.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(self.relu(self.batch_norm3(self.conv3(x))))
        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = CNN().to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Train the Model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).squeeze()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print('Finished Training')

Epoch 1, Loss: 1.4703281702440414
Epoch 2, Loss: 1.0937907459485867
Epoch 3, Loss: 0.9314789445808781
Epoch 4, Loss: 0.8243471680547271
Epoch 5, Loss: 0.7275469566473876
Epoch 6, Loss: 0.6450479019557118
Epoch 7, Loss: 0.5802540668974752
Epoch 8, Loss: 0.5160764171872907
Epoch 9, Loss: 0.45240152352834906
Epoch 10, Loss: 0.4066495546675704
Epoch 11, Loss: 0.3622077407262972
Epoch 12, Loss: 0.32607654590740837
Epoch 13, Loss: 0.28664518040997905
Epoch 14, Loss: 0.2657986928034774
Epoch 15, Loss: 0.23853417199171717
Epoch 16, Loss: 0.21589555784278666
Epoch 17, Loss: 0.20673880149679416
Epoch 18, Loss: 0.19559776469055193
Epoch 19, Loss: 0.17932599193304585
Epoch 20, Loss: 0.16585614362402873
Epoch 21, Loss: 0.15999129952331695
Epoch 22, Loss: 0.15244446119622274
Epoch 23, Loss: 0.14134077527119643
Epoch 24, Loss: 0.1324624614129105
Epoch 25, Loss: 0.13246420795774405
Epoch 26, Loss: 0.12678184928705968
Epoch 27, Loss: 0.12442616249263153
Epoch 28, Loss: 0.11635338375106682
Epoch 29, Los

In [6]:
# Evaluate the Model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).squeeze()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 81.11%
